In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'7vjhjgdc'

In [3]:
wandb.init(id='1bu202z3', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13502.597

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=3e-5)

In [7]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


In [9]:
#wandb.watch(m, log="all")

In [10]:
#train_only_parts_of_model(m, part_to_train="fcs_2")

In [11]:
model_stem = "7.10"
START_E = 0

In [12]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = np.interp(e, [0, 6, 30], [3e-4, 1e-4, 3e-5])
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.00958276, 'trn_headings_loss': 0.00221238, 'trn_curvatures_loss': 0.00544955, 'consistency losses/trn_steer_cost': 0.0210135, 'consistency losses/trn_te_loss': 14.79741097, 'consistency losses/trn_torque_delta_loss': 31.43541282, 'logistical/obs_consumed_per_second': 233.47244094, 'logistical/obs_generated_per_second': 108.36220472, 'logistical/data_consumption_ratio': 2.15138061, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 119.97056537, 'logistical/max_param': 16.23610306, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 235.46875, 'logistical/obs_generated_per_second': 110.0, 'logistical/data_consumption_ratio': 2.14553847, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0090335, 'trn_headings_loss': 0.00213058, 'trn_curvatures_loss': 0.00543879, 'consistency losses/trn_steer_cost': 0.02100241, 'consistency losses/trn_te_loss': 15.13171028, 'consistency losses/trn_torque_delta_loss': 29.6405579

{'logistical/obs_consumed_per_second': 237.84375, 'logistical/obs_generated_per_second': 110.3359375, 'logistical/data_consumption_ratio': 2.15307825, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00851378, 'trn_headings_loss': 0.00188383, 'trn_curvatures_loss': 0.00465241, 'consistency losses/trn_steer_cost': 0.01942122, 'consistency losses/trn_te_loss': 16.38913146, 'consistency losses/trn_torque_loss': 76.07404718, 'consistency losses/trn_torque_delta_loss': 29.80260644, 'logistical/max_param': 16.23989105, 'logistical/lr': 0.00026667}
{'logistical/obs_consumed_per_second': 230.796875, 'logistical/obs_generated_per_second': 106.515625, 'logistical/data_consumption_ratio': 2.16642632, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01011625, 'trn_headings_loss': 0.00202132, 'trn_curvatures_loss': 0.00498377, 'consistency losses/trn_steer_cost': 0.02064532, 'consistency losses/trn_te_loss': 15.117488, 'consistency losses/trn_torque_delta_loss': 32.01283386, '

{'logistical/obs_consumed_per_second': 176.0, 'logistical/obs_generated_per_second': 109.0, 'logistical/data_consumption_ratio': 1.61253619, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 3

{'trn_control_loss': 0.00745016, 'trn_headings_loss': 0.00169414, 'trn_curvatures_loss': 0.00404239, 'consistency losses/trn_steer_cost': 0.01767564, 'consistency losses/trn_te_loss': 16.04299001, 'consistency losses/trn_torque_delta_loss': 26.15354967, 'logistical/obs_consumed_per_second': 225.45669291, 'logistical/obs_generated_per_second': 101.35433071, 'logistical/data_consumption_ratio': 2.23897942, 'logistical/manual_train_pause': 0.00031496, 'consistency losses/trn_torque_loss': 56.56743765, 'logistical/max_param': 16.24244881, 'logistical/lr': 0.0002}
{'logistical/obs_consumed_per_second': 239.3515625, 'logistical/obs_generated_per_second': 100.4296875, 'logistical/data_consumption_ratio': 2.38844991, 'logistical/manual_train_pause': 0.00054688, 'trn_control_loss': 0.00892711, 'trn_

{'logistical/obs_consumed_per_second': 227.828125, 'logistical/obs_generated_per_second': 108.3828125, 'logistical/data_consumption_ratio': 2.10227177, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00784042, 'trn_headings_loss': 0.00184527, 'trn_curvatures_loss': 0.00448394, 'consistency losses/trn_steer_cost': 0.02186024, 'consistency losses/trn_te_loss': 15.62714905, 'consistency losses/trn_torque_delta_loss': 31.73085303, 'consistency losses/trn_torque_loss': 100.33534871, 'logistical/max_param': 16.24816513, 'logistical/lr': 0.00016667}
{'logistical/obs_consumed_per_second': 229.9375, 'logistical/obs_generated_per_second': 105.96875, 'logistical/data_consumption_ratio': 2.17029629, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00795835, 'trn_headings_loss': 0.00190756, 'trn_curvatures_loss': 0.0048399, 'consistency losses/trn_steer_cost': 0.02334267, 'consistency losses/trn_te_loss': 15.44001591, 'consistency losses/trn_torque_delta_loss': 31.39430723, '

{'logistical/obs_consumed_per_second': 233.8515625, 'logistical/obs_generated_per_second': 110.046875, 'logistical/data_consumption_ratio': 2.12736874, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00696819, 'trn_headings_loss': 0.00158724, 'trn_curvatures_loss': 0.00426766, 'consistency losses/trn_steer_cost': 0.01879579, 'consistency losses/trn_te_loss': 15.67982586, 'consistency losses/trn_torque_delta_loss': 28.73240633, 'consistency losses/trn_torque_loss': 88.13625161, 'logistical/max_param': 16.24301147, 'logistical/lr': 0.00013333}
{'logistical/obs_consumed_per_second': 157.0, 'logistical/obs_generated_per_second': 111.0, 'logistical/data_consumption_ratio': 1.41378171, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 6

{'trn_control_loss': 0.0076381, 'trn_headings_loss': 0.00189654, 'trn_curvatures_loss': 0.00482504, 'consistency losses/trn_steer_cost': 0.02186185, 'consistency losses/trn_te_loss': 15.86340436, 'consistency losses/trn_torque_delta_loss': 3

...

wandb: Network error (ConnectTimeout), entering retry loop.


{'logistical/obs_consumed_per_second': 233.6640625, 'logistical/obs_generated_per_second': 106.1875, 'logistical/data_consumption_ratio': 2.20293256, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00638382, 'trn_headings_loss': 0.001514, 'trn_curvatures_loss': 0.00370975, 'consistency losses/trn_steer_cost': 0.0186168, 'consistency losses/trn_te_loss': 15.93415774, 'consistency losses/trn_torque_delta_loss': 25.48805049, 'consistency losses/trn_torque_loss': 78.1070307, 'logistical/max_param': 16.23475075, 'logistical/lr': 3e-05}
{'logistical/obs_consumed_per_second': 147.0, 'logistical/obs_generated_per_second': 101.0, 'logistical/data_consumption_ratio': 1.45292659, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 32

{'trn_control_loss': 0.00669572, 'trn_headings_loss': 0.00160434, 'trn_curvatures_loss': 0.00418326, 'consistency losses/trn_steer_cost': 0.02239311, 'consistency losses/trn_te_loss': 16.46659507, 'consistency losses/trn_torque_delta_loss': 32.0

{'logistical/obs_consumed_per_second': 237.2578125, 'logistical/obs_generated_per_second': 109.984375, 'logistical/data_consumption_ratio': 2.15813302, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00698912, 'trn_headings_loss': 0.00165827, 'trn_curvatures_loss': 0.00411519, 'consistency losses/trn_steer_cost': 0.02238202, 'consistency losses/trn_te_loss': 16.11898804, 'consistency losses/trn_torque_loss': 92.29247626, 'consistency losses/trn_torque_delta_loss': 27.59457865, 'logistical/max_param': 16.23292923, 'logistical/lr': 3e-05}
{'logistical/obs_consumed_per_second': 228.9609375, 'logistical/obs_generated_per_second': 112.796875, 'logistical/data_consumption_ratio': 2.02928149, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00632152, 'trn_headings_loss': 0.00153708, 'trn_curvatures_loss': 0.00383262, 'consistency losses/trn_steer_cost': 0.02017313, 'consistency losses/trn_te_loss': 16.11037528, 'consistency losses/trn_torque_delta_loss': 26.35969435, 'c

{'logistical/obs_consumed_per_second': 226.59375, 'logistical/obs_generated_per_second': 108.3203125, 'logistical/data_consumption_ratio': 2.09575806, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00639547, 'trn_headings_loss': 0.00147329, 'trn_curvatures_loss': 0.00370827, 'consistency losses/trn_steer_cost': 0.02121383, 'consistency losses/trn_te_loss': 16.16419793, 'consistency losses/trn_torque_delta_loss': 27.24095071, 'consistency losses/trn_torque_loss': 73.52081635, 'logistical/max_param': 16.23351479, 'logistical/lr': 3e-05}
{'logistical/obs_consumed_per_second': 157.0, 'logistical/obs_generated_per_second': 112.0, 'logistical/data_consumption_ratio': 1.40383894, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 35

{'trn_control_loss': 0.00660039, 'trn_headings_loss': 0.00162506, 'trn_curvatures_loss': 0.00406116, 'consistency losses/trn_steer_cost': 0.023202, 'consistency losses/trn_te_loss': 15.84758329, 'consistency losses/trn_torque_delta_loss': 2

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e35.torch")

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
